In [1]:
from latentneural.lorenz import LorenzGenerator, ic
from latentneural.data import DataManager
from latentneural.utils import logger
import numpy as np
import tensorflow as tf
import os
import yaml
from copy import deepcopy

In [2]:
step = 0.01 # Time step in the simulatio (in seconds)
stop = 1 # Final time for the simulation (in seconds)
neurons = 30 # Number of neurons
base_rate = 5 # Base frequency (in Hz)
latent_dim = 3 # Lorenz latent variables encoded in neural activity (relevant+irrelevant)
relevant_dim = 2 # Lorenz latent variables encoded in behavioural activity (relevant)
behaviour_dim = 4 # Behavioural channels
conditions = 1 # Number of conditions to be tested (each condition has different weight matrices to convert latent variables into firing rates and behaviour)
trials = 1000 # Number of trials to be tested for each condition
initial_conditions = ic.uniform(low=-10, high=10) # Sample initial conditions of Lorenz system from uniform distribution
behaviour_sigma=1 # Standard deviation noise added to behaviour
seed=12345

In [3]:
with open(os.path.join('..', 'latentneural', 'data', 'storage', \
                       'lorenz', 'grid', 'settings_template.yaml'), 'r') as f:
   training_settings_template = yaml.safe_load(f)
print(training_settings_template)

{'data': {'behavioural_keys': {'test': 'test_behaviours', 'train': 'train_behaviours', 'validation': 'valid_behaviours'}, 'dataset_filename': 'dataset.h5', 'directory': './latentneural/data/storage/lorenz/grid/{{name}}', 'latent_keys': {'test': 'test_latent', 'train': 'train_latent', 'validation': 'valid_latent'}, 'metadata_filename': 'metadata.json', 'neural_keys': {'test': 'test_data', 'train': 'train_data', 'validation': 'valid_data'}}, 'model': {'settings': {'default_layer_settings': {'kernel_initializer': {'arguments': {'distribution': 'normal', 'mode': 'fan_in', 'scale': 1.0}, 'type': 'variance_scaling'}, 'kernel_regularizer': {'arguments': {'l': 0.1}, 'type': 'l2'}}, 'encoded_space': 64, 'layers': {'encoder': {'dropout': 0.05, 'var_max': 0.1, 'var_min': 0.1}, 'decoder': {'kernel_initializer': {'arguments': {'distribution': 'normal', 'mode': 'fan_in', 'scale': 1.0}, 'type': 'variance_scaling'}, 'kernel_regularizer': {'arguments': {'l': 1}, 'type': 'l2'}, 'original_cell': False, '

In [4]:
np.random.seed(seed)
tf.random.set_seed(seed)

for behaviour_sigma in [0.5, 1, 2]:
    for base_rate in [5, 10, 15]:
        for train_num in [50, 100, 200]:
            inner_seed = np.random.randint(0, 2**32 - 1)
            g: LorenzGenerator = LorenzGenerator()
            time, behaviour, spikes, rates, behaviour_w, neural_w, latent = \
                g.generate_spikes_and_behaviour(
                    step=step,
                    stop=stop,
                    n=neurons,
                    base=base_rate,
                    l=latent_dim,
                    b=relevant_dim,
                    y=behaviour_dim,
                    behaviour_sigma=behaviour_sigma,
                    trials=trials,
                    initial_conditions=initial_conditions,
                    conditions=1,
                    seed=inner_seed
                )

            c_i = 0
            generation_settings = dict(
                step=step,
                stop=stop,
                neurons=neurons,
                base_rate=base_rate,
                latent_dim=latent_dim,
                relevant_dim=relevant_dim,
                behaviour_dim=behaviour_dim,
                conditions=conditions,
                behaviour_sigma=behaviour_sigma,
                trials=trials,
                initial_conditions={'type': 'uniform', 'arguments': {'min': -10, 'max': 10}},
                selected_condition=c_i,
                seed=inner_seed)

            valid_num = int(train_num * 0.2)
            test_num = trials - (train_num + valid_num)

            dataset, settings = DataManager.build_dataset(
                neural_data=spikes[c_i,:,:,:],
                behaviour_data=behaviour[c_i,:,:,:],
                settings=generation_settings,
                noisless_behaviour_data=latent[c_i,:,:,-behaviour_w.shape[1]:] @ behaviour_w[c_i,:,:],
                rates_data=rates[c_i,:,:,:],
                latent_data=latent[c_i,:,:,:],
                time_data=time,
                behaviour_weights=behaviour_w[c_i,:,:],
                neural_weights=neural_w[c_i,:,:],
                train_pct=train_num,
                valid_pct=valid_num,
                test_pct=test_num
            )
            dirname = 'train_trials=%d|baselinerate=%d|behaviour_sigma=%.1f' % \
                (train_num, base_rate, behaviour_sigma)
            data_dir = os.path.join('..', 'latentneural', 'data', 'storage', 'lorenz', 'grid', \
                                    dirname)
            DataManager.store_dataset(
                dataset=dataset,
                settings=settings,
                directory=data_dir)
            logger.info('Dataset stored.')
            
            training_settings = deepcopy(training_settings_template)
            training_settings['data']['directory'] = \
                training_settings['data']['directory'].replace('{{name}}', dirname)
            training_settings['output']['directory'] = \
                training_settings['output']['directory'].replace('{{name}}', dirname)
            
            with open(os.path.join(data_dir, 'training_settings.yaml'), 'w') as fp:
                yaml.dump(training_settings, fp)
            logger.info('Training settings stored.')
            logger.info('Stored train_trials=%d|baselinerate=%d|behaviour_sigma=%.1f' % \
                (train_num, base_rate, behaviour_sigma))

[14:43:44.693] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=50|baselinerate=5|behaviour_sigma=0.5/dataset.h5
[14:43:44.705] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_data
[14:43:44.708] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_data
[14:43:44.709] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_data
[14:43:44.729] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours
[14:43:44.731] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[14:43:44.733] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours
[14:43:44.738] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours_noiseless
[14:43:44.740] INFO [latentneural.utils.logging.store

[14:44:01.872] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[14:44:01.874] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours
[14:44:01.879] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours_noiseless
[14:44:01.881] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours_noiseless
[14:44:01.883] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours_noiseless
[14:44:01.889] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_rates
[14:44:01.892] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_rates
[14:44:01.895] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_rates
[14:44:01.923] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_

[14:44:19.220] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_rates
[14:44:19.223] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_rates
[14:44:19.225] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_rates
[14:44:19.255] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_latent
[14:44:19.256] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_latent
[14:44:19.259] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_latent
[14:44:19.263] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: time_data
[14:44:19.264] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: relevant_dims
[14:44:19.266] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: behaviour_weights
[14:44:19.268] INFO [latentneural.utils

[14:44:36.509] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: time_data
[14:44:36.511] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: relevant_dims
[14:44:36.513] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: behaviour_weights
[14:44:36.515] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: neural_weights
[14:44:36.708] INFO [latentneural.utils.logging.<module>:66] Dataset stored.
[14:44:36.717] INFO [latentneural.utils.logging.<module>:76] Training settings stored.
[14:44:36.718] INFO [latentneural.utils.logging.<module>:77] Stored train_trials=50|baselinerate=5|behaviour_sigma=1.0
[14:44:42.139] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=100|baselinerate=5|behaviour_sigma=1.0/dataset.h5
[14:44:42.148] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_dat

[14:44:53.902] INFO [latentneural.utils.logging.<module>:77] Stored train_trials=50|baselinerate=10|behaviour_sigma=1.0
[14:44:59.350] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=100|baselinerate=10|behaviour_sigma=1.0/dataset.h5
[14:44:59.354] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_data
[14:44:59.358] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_data
[14:44:59.360] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_data
[14:44:59.379] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours
[14:44:59.381] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[14:44:59.383] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours
[14:44:59.388] INFO [latentneural.utils.logging.stor

[14:45:16.911] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours
[14:45:16.913] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[14:45:16.915] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours
[14:45:16.919] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours_noiseless
[14:45:16.921] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours_noiseless
[14:45:16.923] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours_noiseless
[14:45:16.927] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_rates
[14:45:16.933] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_rates
[14:45:16.935] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: 

[14:45:40.238] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours_noiseless
[14:45:40.244] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_rates
[14:45:40.248] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_rates
[14:45:40.254] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_rates
[14:45:40.283] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_latent
[14:45:40.289] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_latent
[14:45:40.292] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_latent
[14:45:40.298] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: time_data
[14:45:40.301] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: relevant_dims
[14:45:40.305] INFO [latentneur

[14:45:58.467] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_latent
[14:45:58.471] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: time_data
[14:45:58.472] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: relevant_dims
[14:45:58.474] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: behaviour_weights
[14:45:58.476] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: neural_weights
[14:45:58.660] INFO [latentneural.utils.logging.<module>:66] Dataset stored.
[14:45:58.674] INFO [latentneural.utils.logging.<module>:76] Training settings stored.
[14:45:58.674] INFO [latentneural.utils.logging.<module>:77] Stored train_trials=100|baselinerate=10|behaviour_sigma=2.0
[14:46:04.066] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=200|baselinerate=10|behaviour_sigma=2.0/datas

[14:46:16.074] INFO [latentneural.utils.logging.<module>:76] Training settings stored.
[14:46:16.075] INFO [latentneural.utils.logging.<module>:77] Stored train_trials=100|baselinerate=15|behaviour_sigma=2.0
[14:46:21.761] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=200|baselinerate=15|behaviour_sigma=2.0/dataset.h5
[14:46:21.771] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_data
[14:46:21.779] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_data
[14:46:21.782] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_data
[14:46:21.804] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours
[14:46:21.806] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[14:46:21.809] INFO [latentneural.utils.logging.store_dataset:204] Saving 